In [33]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [34]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Thu, 23 Jul 2020 02:54:41 GMT', 'Server': 'mw1265.eqiad.wmnet', 'X-Content-Type-Options': 'nosniff', 'P3p': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-Language': 'en', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'X-Request-Id': '628d4537-c294-4879-8495-90635801e84b', 'Last-Modified': 'Thu, 23 Jul 2020 02:54:05 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Encoding': 'gzip', 'Age': '30170', 'X-Cache': 'cp1081 hit, cp1089 hit/19', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=23-Jul-2020;Path=/;HttpOnly;secure;Expires=Mon, 24 Aug 2020 00:00:00 GMT, WMF-Last-Access-Global=23-Jul-2020;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Mon, 24 Aug 2020 00:00:00 GMT, GeoIP=CA:ON:Markham:43.88:-79.26:v4; Path=/;

In [35]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])
#lst

In [36]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
tab1 = pd.DataFrame(lst, columns=cols)
print(tab1.shape)

(103, 3)


In [38]:
tab1 = tab1.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

In [39]:
tab1.loc[tab1['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [40]:
tab1.shape

(103, 3)

In [41]:
tab2 = pd.read_csv("http://cocl.us/Geospatial_data")
tab2.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [42]:
tab2 = pd.merge(df, dfgeo, on="PostalCode", how='left')

In [43]:
tab2.loc[df2['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [44]:
tab2.loc[df2['PostalCode'] == 'M9V']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [45]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(tab2['Latitude'], tab2['Longitude'], tab2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [46]:
CLIENT_ID = '3Y2UIXY513PP1Z4BP0Z5LEQZP1RQXTY5N1KL5L33NQXIHSJS' # Foursquare ID
CLIENT_SECRET = '4IJ0F2MIVMNBA3EMF1JSTDF10JWSB5UZ2UF2VQKEDVAG4JLX' # Foursquare Secret
VERSION = '20180605' # API version

In [47]:
tab2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = tab2.loc['M9V']['Latitude']
neighborhood_longitude = tab2.loc['M9V']['Longitude']

In [48]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=3Y2UIXY513PP1Z4BP0Z5LEQZP1RQXTY5N1KL5L33NQXIHSJS&client_secret=4IJ0F2MIVMNBA3EMF1JSTDF10JWSB5UZ2UF2VQKEDVAG4JLX&v=20180605&ll=43.739416399999996,-79.5884369&radius=500&limit=100'

In [50]:
results = requests.get(url).json()
#results

In [51]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [52]:
venues = results['response']['groups'][0]['items']

In [53]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups
0,e-0-4be58dc4cf200f479154133c-0,0,"[{'summary': 'This spot is popular', 'type': '...",4be58dc4cf200f479154133c,Shoppers Drug Mart,1530 Albion Rd,Albion Mall,43.741685,-79.584487,"[{'label': 'display', 'lat': 43.741685, 'lng':...",405,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (Albion Mall), Etobicoke ON M9...","[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",0,[]
1,e-0-4c633939e1621b8d48842553-1,0,"[{'summary': 'This spot is popular', 'type': '...",4c633939e1621b8d48842553,Subway,"6210 Finch Ave West, Store 103",at Albion Rd.,43.742645,-79.589643,"[{'label': 'display', 'lat': 43.74264512142215...",372,M9V 0A1,CA,Toronto,ON,Canada,"[6210 Finch Ave West, Store 103 (at Albion Rd....","[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",0,[]
2,e-0-4be70e26cf200f47e334153c-2,0,"[{'summary': 'This spot is popular', 'type': '...",4be70e26cf200f47e334153c,Popeyes Louisiana Kitchen,1530 Albion Rd.,at Kipling Ave. (Albion Centre),43.741202,-79.584545,"[{'label': 'display', 'lat': 43.74120165509377...",370,M9V 1B4,CA,Etobicoke,ON,Canada,[1530 Albion Rd. (at Kipling Ave. (Albion Cent...,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",0,[]
3,e-0-5112b872e4b0c0a78d7bcd27-3,0,"[{'summary': 'This spot is popular', 'type': '...",5112b872e4b0c0a78d7bcd27,Sunny Foodmart,1620 Albion road,Albion Road and Finch Ave,43.741840,-79.590561,"[{'label': 'display', 'lat': 43.74184023292111...",319,NaN,CA,Toronto,ON,Canada,"[1620 Albion road (Albion Road and Finch Ave),...","[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[]
4,e-0-4cd4738cdfb4a1cd4337535c-4,0,"[{'summary': 'This spot is popular', 'type': '...",4cd4738cdfb4a1cd4337535c,The Beer Store,1530 Albion Rd,Near Finch Ave. W.,43.741694,-79.584373,"[{'label': 'display', 'lat': 43.7416936, 'lng'...",413,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (Near Finch Ave. W.), Etobicok...","[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",0,[]
5,e-0-4d8ba6910c4e41bdaaf7667f-5,0,"[{'summary': 'This spot is popular', 'type': '...",4d8ba6910c4e41bdaaf7667f,Pizza Pizza,"1530 Albion Road, Unit T25",NaN,43.741569,-79.584489,"[{'label': 'display', 'lat': 43.74156896801906...",397,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Road, Unit T25, Etobicoke ON M9V ...","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[]
6,e-0-4b04a05bf964a520c45522e3-6,0,"[{'summary': 'This spot is popular', 'type': '...",4b04a05bf964a520c45522e3,Sheriff's No Frills,1530 Albion Rd,at Finch Ave. W.,43.741696,-79.584379,"[{'label': 'display', 'lat': 43.741696, 'lng':...",413,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (at Finch Ave. W.), Etobicoke ...","[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[]
7,e-0-4c1951d6834e2d7f2d3a2a80-7,0,"[{'summary': 'This spot is popular', 'type': '...",4c1951d6834e2d7f2d3a2a80,McDonald's,"1530 Albion Road, Unit F-1",NaN,43.741757,-79.584230,"[{'label': 'display', 'lat': 43.7417571, 'lng'...",427,M9V 1B4,CA,Toronto,ON,Canada,"[1530 Albion Road, Unit F-1, Toronto ON M9V 1B...","[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",0,[]
8,e-0-4da73b855da37feee89a1143-8,0,"[{'summary': 'This spot is popular', 'type': '...",4da73b855da37feee89a1143,Pizza Nova,1701 Martingrove Road,NaN,43.736761,-79.589817,"[{'label': 'display', 'lat': 43.73676056243852...",315,M9V 4N4,CA,Toronto,ON,Canada,"[1701 Martingrove Road, Toronto ON M9V 4N4, Ca...","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[]


In [54]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Shoppers Drug Mart,Pharmacy,43.741685,-79.584487
1,Subway,Sandwich Place,43.742645,-79.589643
2,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.741202,-79.584545
3,Sunny Foodmart,Grocery Store,43.741840,-79.590561
4,The Beer Store,Beer Store,43.741694,-79.584373


In [55]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


In [56]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
          

In [57]:
tab2

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...
M9N,York,Weston,43.706876,-79.518188
M9P,Etobicoke,Westmount,43.696319,-79.532242
M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724


In [58]:
venues = getNearbyVenues(names=tab2['Borough'],latitudes=tab2['Latitude'],longitudes=tab2['Longitude'])

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [59]:
print(venues.shape)
venues.head()

(2153, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
2,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Scarborough,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,Scarborough,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [60]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,120,120,120,120,120,120
Downtown Toronto,1243,1243,1243,1243,1243,1243
East Toronto,124,124,124,124,124,124
East York,75,75,75,75,75,75
Etobicoke,72,72,72,72,72,72
Mississauga,13,13,13,13,13,13
North York,236,236,236,236,236,236
Scarborough,90,90,90,90,90,90
West Toronto,162,162,162,162,162,162


In [61]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 269 unique categories.


In [62]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
onehot.shape

(2153, 269)

In [65]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008333,...,0.016667,0.000000,0.008333,0.000000,0.008333,0.000000,0.000000,0.000000,0.000000,0.008333
1,Downtown Toronto,0.000000,0.000805,0.000805,0.000805,0.000805,0.001609,0.002414,0.001609,0.013677,...,0.000805,0.002414,0.011263,0.001609,0.003218,0.000000,0.006436,0.000000,0.000805,0.006436
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024194,...,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.008065,0.000000,0.000000,0.024194
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.013333,0.000000,0.000000,0.000000,0.013333
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013889,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013889,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004237,0.000000,0.004237,0.000000,0.000000,0.000000,0.000000,0.000000,0.008475,...,0.000000,0.000000,0.000000,0.004237,0.012712,0.000000,0.000000,0.000000,0.004237,0.000000
7,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011111,...,0.000000,0.000000,0.000000,0.000000,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000
8,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.018519,0.000000,0.012346,0.000000,0.006173,0.000000,0.000000,0.012346
9,York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000


In [66]:
grouped.shape

(10, 269)

In [67]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Park  0.06
3     Pizza Place  0.06
4            Café  0.05


----Downtown Toronto----
                venue  freq
0         Coffee Shop  0.11
1                Café  0.05
2          Restaurant  0.03
3               Hotel  0.03
4  Seafood Restaurant  0.02


----East Toronto----
                venue  freq
0    Greek Restaurant  0.06
1         Coffee Shop  0.05
2             Brewery  0.04
3  Italian Restaurant  0.04
4                Café  0.04


----East York----
          venue  freq
0          Bank  0.05
1   Coffee Shop  0.05
2          Park  0.05
3      Pharmacy  0.04
4  Burger Joint  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.12
1  Sandwich Place  0.07
2        Pharmacy  0.06
3     Coffee Shop  0.06
4   Grocery Store  0.04


----Mississauga----
                      venue  freq
0                     Hotel  0.15
1               Coffee Shop  0.15
2     

In [68]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [69]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Pizza Place,Park,Sandwich Place,Café,Sushi Restaurant,Pub,Restaurant,Dessert Shop,Clothing Store
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Gym,Bakery,Clothing Store
2,East Toronto,Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Café,Park,Restaurant,Ice Cream Shop,Yoga Studio,Pizza Place
3,East York,Park,Bank,Coffee Shop,Sandwich Place,Pizza Place,Pharmacy,Burger Joint,Sporting Goods Shop,Liquor Store,Supermarket
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Gym,Grocery Store,Fast Food Restaurant,Bakery,Discount Store,Business Service
5,Mississauga,Coffee Shop,Hotel,Gym,Gas Station,Mediterranean Restaurant,Sandwich Place,Burrito Place,Middle Eastern Restaurant,Intersection,Fried Chicken Joint
6,North York,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Pizza Place,Sandwich Place,Bank,Park,Grocery Store,Fast Food Restaurant
7,Scarborough,Chinese Restaurant,Breakfast Spot,Bank,Bakery,Fast Food Restaurant,Coffee Shop,Skating Rink,Pizza Place,Discount Store,Mexican Restaurant
8,West Toronto,Café,Bar,Coffee Shop,Bakery,Italian Restaurant,Restaurant,Grocery Store,Pizza Place,Breakfast Spot,Bookstore
9,York,Convenience Store,Park,Playground,Caribbean Restaurant,Coffee Shop,Women's Store,Discount Store,Brewery,Trail,Field


In [70]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 3, 4, 2, 3, 3, 0, 1], dtype=int32)

In [71]:
merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008333,...,Coffee Shop,Pizza Place,Park,Sandwich Place,Café,Sushi Restaurant,Pub,Restaurant,Dessert Shop,Clothing Store
1,Downtown Toronto,0.0,0.000805,0.000805,0.000805,0.000805,0.001609,0.002414,0.001609,0.013677,...,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Gym,Bakery,Clothing Store
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024194,...,Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Café,Park,Restaurant,Ice Cream Shop,Yoga Studio,Pizza Place
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Park,Bank,Coffee Shop,Sandwich Place,Pizza Place,Pharmacy,Burger Joint,Sporting Goods Shop,Liquor Store,Supermarket
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013889,...,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Gym,Grocery Store,Fast Food Restaurant,Bakery,Discount Store,Business Service
